In [1]:
!pip install rdflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 500 kB 31.5 MB/s 
     |████████████████████████████████| 41 kB 672 kB/s 


In [2]:
from google.colab import drive

from rdflib import Graph, Literal, BNode, Namespace, URIRef
from rdflib.namespace import XSD, RDF, OWL, RDFS, VOID

drive.mount('/content/gdrive')

ontology_url = 'gdrive/My Drive/Colab Notebooks/OntologiesRepresentationKnowledge/filled-tanks-ontology.owl'

Mounted at /content/gdrive


In [3]:
# Create a Graph
tanks_ontology_graph = Graph()

# Parse in an RDF file
tanks_ontology_graph.parse(ontology_url, format="turtle")

print(len(tanks_ontology_graph))

4595028


In [4]:
# 1. Сколько игроков в бою превысили свои средние показатели по урону на используемом танке?
player_success_query = """
  PREFIX : <http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  SELECT (COUNT(?player) as ?superiorPlayers) WHERE {
    ?battleResults a :BattleResults .
    ?battleResults :isResultsOfPlayer ?player .
    ?battleResults :playedOn ?tank .
    ?battleResults :batResDamage ?damageInBattle .
    ?playerStatOnTank :tankStatisticOf ?player .
    ?playerStatOnTank :tankStatisticOn ?tank .
    ?playerStatOnTank :concreteTankDamageDealt ?damageDealt .
    ?playerStatOnTank :concreteTankBattles ?battlesOnTank .
    FILTER (xsd:float(?damageInBattle) > ?damageDealt/?battlesOnTank)
  }
"""

for row in tanks_ontology_graph.query(player_success_query):
    print(f"{row.superiorPlayers} of players surpassed themselves")

85 of players surpassed themselves


In [51]:
# 2. Какой из танков, представленных в бою, имеется у наибольшего числа игроков в статистике?
most_popular_tank_query = """
  PREFIX : <http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  SELECT (COUNT(?tankName) as ?count) WHERE {
    ?battleResults a :BattleResults .
    ?battleResults :isResultsOfPlayer ?player .
    ?battleResults :playedOn ?tank .
    ?tank :vechicleName ?tankName .
  }
"""

for row in tanks_ontology_graph.query(most_popular_tank_query):
    print(f"count {row.count}")

count <built-in method count of ResultRow object at 0x7ff177af5440>


In [63]:
# 3. Какой самый популярный тип техники по сумме сыгранных боёв среди игроков?
most_playable_tank_query = """
  PREFIX : <http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  SELECT ?tankType (SUM(?totalBattles) AS ?totalSum) WHERE {
    ?statistic a :PlayerStatisticOnConcreteTank .
    ?statistic :tankStatisticOn ?tank .
    ?tank :hasType ?tankType .
    ?statistic :concreteTankBattles ?totalBattles .
  }
  GROUP BY ?tankType
  ORDER BY DESC(?totalSum)
  LIMIT 1
"""

for row in tanks_ontology_graph.query(most_playable_tank_query):
    print(f"tankType {row.tankType} totalSum {row.totalSum}")

tankType http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#heavyTank totalSum 10737643


In [38]:
# 4. Тяжелые танки какой нации наименее результативны?
nation_of_worst_heavy_tank_query = """
  PREFIX : <http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  SELECT ?nation (AVG(xsd:float(?wins)/xsd:float(?battles)) AS ?avgWinPercen) WHERE {
    ?tank a :Tank.
    ?tank :hasType :heavyTank .
    ?tank :hasNation ?nation .
    ?statistic :tankStatisticOn ?tank .
    ?statistic :concreteTankWins ?wins .
    ?statistic :concreteTankBattles ?battles .
  }
  GROUP BY ?nation
  ORDER BY DESC(?avgWinPercen)
  LIMIT 1
"""

for row in tanks_ontology_graph.query(nation_of_worst_heavy_tank_query):
    print(f"nation {row.nation} avgWinPercen {row.avgWinPercen}")

nation http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#china avgWinPercen 0.5108836515555025


In [8]:
# 5. Кто повалил больше всего деревьев в среднем за бой?
max_treesCut_query = """
  PREFIX : <http://www.semanticweb.org/ko4ebhuk/ontologies/2022/10/tanks#>
  PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
  SELECT ?name (xsd:float(?treesCut)/xsd:float(?battlesCount) AS ?result) WHERE {
    ?player a :Player .
    ?player :nickName ?name .
    ?player :hasStatistic ?statistic .
    ?statistic :treesCut ?treesCut .
    ?statistic :battles ?battlesCount .
  }
  ORDER BY DESC(?result)
  LIMIT 1
"""

for row in tanks_ontology_graph.query(max_treesCut_query):
    print(f"{row.name} avg_cuts {row.result}")

15_MunyT_ADA avg_cuts 3.426887094707164


In [64]:
# VoID
from rdflib.void import generateVoID

void = generateVoID(tanks_ontology_graph)

In [66]:
void[0].serialize(destination="status.owl")

<Graph identifier=N4502729e99d0419cb0965ec21373b490 (<class 'rdflib.graph.Graph'>)>